<div style="background-color:#000;"><img src="pqn.png"></img></div><div><a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.</div>

## Library installation

Install numpy and matplotlib so the notebook can simulate and plot GBM paths. This ensures a clean, reproducible environment regardless of your local setup.

In [ ]:
!pip install numpy matplotlib

If you are running in an environment that already ships these libraries (such as Colab or some IDEs), this command will simply confirm versions. Keeping dependencies minimal lets us focus on modeling choices like time-step units and volatility scaling rather than environment issues.

## Imports and setup

We use numpy for fast vectorized arrays and random draws, and matplotlib.pyplot for quick visual checks of simulated price paths.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Visual checks are a core part of simulation work because they surface unit mismatches early. If you want reproducible runs later, you can set a seed with numpy’s RNG before calling any functions; reproducibility is essential when diagnosing differences across parameter choices.

## Define simulation parameters and units

Set the initial price, annualized volatility, and annualized drift, then define the Monte Carlo grid (paths, daily time step, total horizon). Using delta = 1/252 converts annual inputs into daily increments, and time = 252*5 targets a five-year window.

In [ ]:
s0 = 131.00
sigma = 0.25
mu = 0.35
paths = 1000
delta = 1.0 / 252.0
time = 252 * 5

Locking units up front prevents the classic mistake of feeding annual parameters into daily steps without the proper scaling. Consistent definitions make results interpretable and comparable as you vary drift or volatility. paths controls Monte Carlo precision, so increasing it is how we check convergence of any statistic we care about.

## Build GBM and helper functions

Generate Brownian shocks whose variance matches the chosen time step and whose scale reflects the (annualized) volatility. Each column is an independent price path, and rows step forward in time.

In [ ]:
def wiener_process(delta, sigma, time, paths):
    """Returns a Wiener process.

    Parameters
    ----------
    delta : float
        The increment to downsample sigma.
    sigma : float
        Percentage volatility.
    time : int
        Number of samples to create.
    paths : int
        Number of price simulations to create.

    Returns
    -------
    wiener_process : np.ndarray

    Notes
    -----
    This method returns a Wiener process. The Wiener process is also called
    Brownian motion. For more information about the Wiener process check out
    the Wikipedia page:
    http://en.wikipedia.org/wiki/Wiener_process
    """
    return sigma * np.random.normal(
        loc=0,
        scale=np.sqrt(delta),
        size=(time, paths),
    )

Scaling by sqrt(delta) gives the correct time-step variance, and multiplying by sigma downscales the annual volatility to the per-step level. This independence-by-path assumption is the default baseline in pricing engines and is ideal for sandbox learning. Vectorized generation keeps the simulation fast enough to run thousands of paths interactively.

Convert Brownian shocks into multiplicative GBM returns using the closed-form step of a constant-drift, constant-vol process. The (mu - 0.5*sigma**2)*delta term applies the Itô correction so expected log returns line up with the specified drift.

In [ ]:
def gbm_returns(delta, sigma, time, mu, paths):
    """Returns from a Geometric Brownian Motion (GBM).

    Parameters
    ----------
    delta : float
        The increment to downsample sigma.
    sigma : float
        Percentage volatility.
    time : int
        Number of samples to create.
    mu : float
        Percentage drift.
    paths : int
        Number of price simulations to create.

    Returns
    -------
    gbm_returns : np.ndarray

    Notes
    -----
    This method constructs random Geometric Brownian Motion (GBM).
    """
    process = wiener_process(delta, sigma, time, paths)
    return np.exp(process + (mu - sigma**2 / 2) * delta)

Many beginners omit the 0.5*sigma^2 term and end up with biased levels that drift too high. Keeping drift and volatility annualized while applying delta per step is the key to unit consistency. This block encodes those rules in one place so we can change parameters confidently.

Compound the simulated returns from the initial price to produce level paths while ensuring strictly positive prices. We prepend ones so the first row represents the starting point before any move.

In [ ]:
def gbm_levels(s0, delta, sigma, time, mu, paths):
    """Returns price paths starting at s0.

    Parameters
    ----------
    s0 : float
        The starting stock price.
    delta : float
        The increment to downsample sigma.
    sigma : float
        Percentage volatility.
    time : int
        Number of samples to create.
    mu : float
        Percentage drift.
    paths : int
        Number of price simulations to create.

    Returns
    -------
    gbm_levels : np.ndarray
    """
    returns = gbm_returns(delta, sigma, time, mu, paths)
    stacked = np.vstack([np.ones(paths), returns])
    return s0 * stacked.cumprod(axis=0)

Working with levels is how option desks and risk teams visualize scenarios and payoffs. GBM’s multiplicative compounding matches how real prices behave and avoids negative levels that plague additive models. Including the initial price makes plots and summary stats easier to interpret.

## Run scenarios and visualize paths

Simulate paths with a positive drift and compute how many finishes end above the starting price as a quick sanity check. In notebooks this last expression will display and gives a rough sense of the distribution’s tilt under positive drift.

In [ ]:
price_paths = gbm_levels(s0, delta, sigma, time, mu, paths)
len(price_paths[-1, price_paths[-1, :] > s0])

This quick diagnostic connects drift to long-run tendency without overfitting historical quirks. As you scale paths higher, the fraction should stabilize, which is a simple convergence check. It’s a habit pros use before trusting any simulated payoff estimates.

Plot the simulated paths to visually verify unit choices and compounding behavior. Thin lines help us inspect many paths at once.

In [ ]:
plt.plot(price_paths, linewidth=0.25)
plt.show()

Early plots catch mistakes like mismatched time steps or mis-scaled volatility before you burn time on strategy code. Visual checks pair with seeded randomness to make discrepancies obvious when you tweak parameters. Treat these figures as fast feedback loops, not final analyses.

Rerun the engine with zero drift to isolate volatility’s effect and compare to the positive-drift case. Visualizing both back-to-back reinforces how drift sets direction while volatility controls dispersion.

In [ ]:
price_paths = gbm_levels(s0, delta, sigma, time, 0.0, paths)
plt.plot(price_paths, linewidth=0.25)
plt.show()

A zero-drift scenario is a common baseline in pricing and risk because it mirrors a risk-neutral setting for intuition-building. You should see no systematic trend in log space, yet level paths remain positively skewed due to compounding. Use this contrast to validate that your step size and scaling behave as expected.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.